In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9948354276776387, 0.9946733190391692)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9978803639087322, 0.9981102023478097)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [9]:
y

,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,NaN,Region,NaN,Year,...,Name,Surname,NaN,NaN,NaN,NaN,Date,BiologicalSex,Boolean,NaN
1,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,NaN,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
2,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,NaN,Year,...,Name,Surname,String,NaN,NaN,Currency,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,CadastreCode,Document,Plate,Error,ItalianZIPCode,ProvinceCode,Region,Municipality,NaN,...,NaN,Surname,NaN,EMail,PhoneNumber,Currency,NaN,BiologicalSex,NaN,NumericId
4,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,NaN,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
5,NaN,CadastreCode,Document,Plate,Address,NaN,ProvinceCode,Error,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,NaN,Boolean,NumericId
6,Error,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,NaN,...,Error,Surname,NaN,EMail,PhoneNumber,Error,Date,BiologicalSex,Boolean,NumericId
7,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Name,NaN,String,EMail,PhoneNumber,Error,Date,BiologicalSex,NaN,NumericId
8,ItalianFiscalCode,CadastreCode,NaN,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,NaN,NaN,BiologicalSex,NaN,NumericId
9,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,NaN,...,Error,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NaN


In [11]:
model.predict_dataframe(X)

,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,NaN,Region,NaN,Year,...,Name,Surname,NaN,NaN,NaN,NaN,Date,BiologicalSex,Boolean,NaN
1,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,NaN,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
2,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,NaN,Year,...,Name,Surname,String,NaN,NaN,Currency,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,CadastreCode,Document,Plate,Error,ItalianZIPCode,ProvinceCode,Region,Municipality,NaN,...,NaN,Surname,NaN,EMail,PhoneNumber,Currency,NaN,BiologicalSex,NaN,NumericId
4,ItalianFiscalCode,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,NaN,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
5,NaN,CadastreCode,Document,Plate,Address,NaN,ProvinceCode,Error,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,NaN,Boolean,NumericId
6,Error,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,NaN,...,Error,Surname,NaN,EMail,PhoneNumber,Error,Date,BiologicalSex,Boolean,NumericId
7,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Name,NaN,String,EMail,PhoneNumber,Error,Date,BiologicalSex,NaN,NumericId
8,ItalianFiscalCode,CadastreCode,NaN,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,Name,Surname,String,EMail,PhoneNumber,NaN,NaN,BiologicalSex,NaN,NumericId
9,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,NaN,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NaN


In [18]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test)
predicted_labels = model._embedder._encoder.inverse_transform(y_pred)

#Counter(zip(true_labels, predicted_labels))

In [20]:
((true_labels == "Error") & (predicted_labels == "Address")).shape

(1226239,)

In [21]:
X.size

744